## Part I - Bayesian networks

In this part we will consider Directed Acyclic Graphs to model our financial environment with conditional densities.

In [1]:
import pandas as pd
import numpy as np
import itertools

### Data processing

In [2]:
market = pd.read_csv("data/market.csv")
mapping = pd.read_csv("data/map.csv")

sector_of = dict(zip(mapping["asset"], mapping["sector"]))

market_var = "MKT"
assets = ['A001', 'A002', 'A003', 'A004', 'A005', 'A011', 'A012', 'A013', 'A021'] # ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A051', 'A052', 'A053', 'A054', 'A055', 'A056', 'A057', 'A058', 'A059', 'A060', 'A081', 'A082', 'A083', 'A084', 'A085', 'A086', 'A087', 'A088', 'A089', 'A090']
sectors = sorted(list(set([sector_of[asset] for asset in assets])))
sector_assets = {sector: [asset for asset in sector_of.keys() if sector_of[asset] == sector] for sector in sectors}

column_filter = [market_var] + assets + sectors 
data = market[column_filter]


# Creation of a dynamic dataset (time t and t-1)

data_t = data.iloc[1:].reset_index(drop=True)
data_tm1 = data.iloc[:-1].reset_index(drop=True)
data_tm1.columns = [c + "_lag" for c in data_tm1.columns]

data = pd.concat([data_t, data_tm1], axis=1)
n_samples = len(data)
data

,MKT,A001,A002,A003,A004,A005,A011,A012,A013,A021,...,A003_lag,A004_lag,A005_lag,A011_lag,A012_lag,A013_lag,A021_lag,SEC01_lag,SEC02_lag,SEC03_lag
0,0.004882,0.004498,0.014087,0.019456,0.010936,0.016938,-0.011931,-0.010511,0.001958,-0.027173,...,0.020042,0.007944,0.012824,0.007827,0.005984,0.004566,0.007621,0.013723,0.006033,0.012443
1,-0.008071,-0.006948,0.005399,0.013493,0.022757,-0.004369,-0.027028,-0.006248,-0.042334,-0.007275,...,0.019456,0.010936,0.016938,-0.011931,-0.010511,0.001958,-0.027173,0.015384,-0.001119,-0.003761
2,0.000945,0.022392,0.029333,0.018555,0.017607,0.019644,-0.007575,-0.016549,0.004376,0.019181,...,0.013493,0.022757,-0.004369,-0.027028,-0.006248,-0.042334,-0.007275,-0.003830,-0.017498,-0.006121
3,0.001240,-0.002762,-0.006140,-0.008366,-0.002357,-0.018107,-0.001741,-0.021279,-0.009492,-0.000398,...,0.018555,0.017607,0.019644,-0.007575,-0.016549,0.004376,0.019181,0.019916,-0.014134,-0.006198
4,0.012147,0.004508,0.004004,-0.003563,-0.016581,0.009945,0.005026,0.011135,0.001254,0.001123,...,-0.008366,-0.002357,-0.018107,-0.001741,-0.021279,-0.009492,-0.000398,-0.006248,0.005246,-0.005167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,-0.016966,-0.033545,-0.005110,-0.029102,-0.029927,-0.039487,-0.021183,-0.044509,-0.027983,-0.001965,...,0.045760,-0.011367,0.007738,0.007855,0.018420,0.008138,0.003164,0.012671,0.006500,-0.021637
995,0.001483,0.012957,-0.005558,0.029565,0.000915,-0.002478,0.003806,0.018548,0.025410,-0.005247,...,-0.029102,-0.029927,-0.039487,-0.021183,-0.044509,-0.027983,-0.001965,-0.037978,-0.015342,-0.017327
996,-0.012265,0.009064,0.025532,-0.022437,-0.002858,-0.019700,-0.024078,0.000183,-0.022712,-0.009668,...,0.029565,0.000915,-0.002478,0.003806,0.018548,0.025410,-0.005247,0.005064,0.007624,-0.007903
997,-0.008496,0.006281,-0.004436,0.003306,-0.026351,-0.011068,-0.019958,-0.039244,-0.010202,0.004437,...,-0.022437,-0.002858,-0.019700,-0.024078,0.000183,-0.022712,-0.009668,-0.004598,-0.018096,0.003126


### Structure learning based on Hill Climb algorithm

To model our Bayesian Network, we consider a dynamic framework where we allow for time dependencies between assets.

In [3]:
def bic_score(y, X):
    n = len(y)

    if X.shape[1] == 0:
        resid = y - y.mean()
        var = np.var(resid)
        ll = -0.5 * n * (np.log(2 * np.pi * var) + 1)
        k = 1
    else:
        beta = np.linalg.lstsq(X, y, rcond=None)[0]
        resid = y - X @ beta
        var = np.var(resid)
        ll = -0.5 * n * (np.log(2 * np.pi * var) + 1)
        k = X.shape[1] + 1

    return ll - 0.5 * k * np.log(n)

fixed_edges = []
candidate_edges = []
candidate_edges.append((market_var + "_lag", market_var))

for sec in sectors:
    candidate_edges.append((market_var, sec))
    candidate_edges.append((sec + "_lag", sec))

# SEC -> A
# A(t-1) -> A(t)
for a in assets:
    candidate_edges.append((market_var, a))
    candidate_edges.append((sector_of[a], a))
    candidate_edges.append((a + "_lag", a))


def total_score(edges):
    score = 0.0
    for var in data_t.columns:
        parents = [p for (p, c) in edges if c == var]
        if len(parents) == 0:
            X = np.empty((n_samples, 0))
        else:
            X = data[parents].values
        y = data[var].values
        score += bic_score(y, X)
    return score


edges = fixed_edges.copy()
current_score = total_score(edges)

improved = True

while improved:
    improved = False
    best_candidate = None
    best_score = current_score

    for e in candidate_edges:
        if e in edges:
            continue

        trial_edges = edges + [e]

        children = [c for (_, c) in trial_edges]
        if len(children) != len(set(children)):
            continue

        s = total_score(trial_edges)

        if s > best_score:
            best_score = s
            best_candidate = e

    if best_candidate is not None:
        edges.append(best_candidate)
        current_score = best_score
        improved = True
        print(f"Adding edge {best_candidate}, score = {current_score:.2f}")


print("\n Final structure of DBN :\n")
for e in edges:
    print(f"{e[0]} -> {e[1]}")

Adding edge ('SEC01', 'A001'), score = 37498.41
Adding edge ('MKT', 'SEC01'), score = 37974.77
Adding edge ('SEC01', 'A003'), score = 38432.93
Adding edge ('SEC02', 'A013'), score = 38883.21
Adding edge ('SEC02', 'A011'), score = 39326.20
Adding edge ('SEC01', 'A005'), score = 39639.53
Adding edge ('MKT', 'SEC02'), score = 39940.86
Adding edge ('SEC02', 'A012'), score = 40215.34
Adding edge ('MKT', 'SEC03'), score = 40450.71
Adding edge ('SEC01', 'A004'), score = 40665.18
Adding edge ('SEC01', 'A002'), score = 40817.94
Adding edge ('SEC03', 'A021'), score = 40965.03

 Final structure of DBN :

SEC01 -> A001
MKT -> SEC01
SEC01 -> A003
SEC02 -> A013
SEC02 -> A011
SEC01 -> A005
MKT -> SEC02
SEC02 -> A012
MKT -> SEC03
SEC01 -> A004
SEC01 -> A002
SEC03 -> A021


### Structure learning with PC algorithm

In [4]:
def is_asset(var):
    return var.startswith("A")

def is_sector(var):
    return var.startswith("SEC")

def is_market(var):
    return var == "MKT"

def is_lag(var):
    return var.endswith("_lag")

In [5]:
from scipy.stats import norm

def partial_correlation_test(data, x, y, Z):
    """
    Test gaussien : x ⫫ y | Z
    Retourne la p-value
    """
    X = data[[x]].values
    Y = data[[y]].values

    if len(Z) == 0:
        r = np.corrcoef(X[:, 0], Y[:, 0])[0, 1]
    else:
        Zm = data[list(Z)].values

        beta_x = np.linalg.lstsq(Zm, X, rcond=None)[0]
        beta_y = np.linalg.lstsq(Zm, Y, rcond=None)[0]

        rx = X - Zm @ beta_x
        ry = Y - Zm @ beta_y

        r = np.corrcoef(rx[:, 0], ry[:, 0])[0, 1]

    r = np.clip(r, -0.999, 0.999)
    n = data.shape[0]
    z = 0.5 * np.log((1 + r) / (1 - r))
    stat = np.sqrt(n - len(Z) - 3) * abs(z)
    pval = 2 * (1 - norm.cdf(stat))

    return pval


def edge_allowed(x, y):
    if is_lag(x) and is_lag(y):
        return False

    if is_asset(x) and is_asset(y):
        if not is_lag(x) and not is_lag(y):
            return False

    if is_asset(x) and is_sector(y):
        return False

    if is_asset(x) and is_lag(x) and is_sector(y):
        return False
    return True


def pc_skeleton(data, alpha=0.05, max_cond_set=3):
    nodes = list(data.columns)
    adj = {x: set(nodes) - {x} for x in nodes}
    sep_set = {}

    l = 0
    while True:
        cont = False
        for x, y in itertools.combinations(nodes, 2):
            if not edge_allowed(x, y):
                continue
            if y not in adj[x]:
                continue

            neighbors = adj[x] - {y}
            if len(neighbors) < l:
                continue

            for Z in itertools.combinations(neighbors, l):
                pval = partial_correlation_test(data, x, y, Z)
                if pval > alpha:
                    adj[x].remove(y)
                    adj[y].remove(x)
                    sep_set[(x, y)] = set(Z)
                    sep_set[(y, x)] = set(Z)
                    cont = True
                    break
        if not cont or l >= max_cond_set:
            break
        l += 1

    return adj, sep_set


def orient_colliders(adj, sep_set):
    directed = set()
    undirected = set()

    for x in adj:
        for y in adj[x]:
            if edge_allowed(x, y):
                undirected.add((x, y))

    nodes = list(adj.keys())

    for x, z, y in itertools.permutations(nodes, 3):
        if not (edge_allowed(x, z) and edge_allowed(y, z)):
            continue

        if z in adj[x] and z in adj[y] and x not in adj[y]:
            if z not in sep_set.get((x, y), set()):
                directed.add((x, z))
                directed.add((y, z))
                undirected.discard((x, z))
                undirected.discard((z, x))
                undirected.discard((y, z))
                undirected.discard((z, y))

    return directed, undirected




def apply_meek_rules(directed, undirected):
    changed = True
    while changed:
        changed = False

        for a, b in list(directed):
            for c, d in list(undirected):
                if b == c and (a, d) not in directed and (d, a) not in directed:
                    directed.add((b, d))
                    undirected.discard((c, d))
                    undirected.discard((d, c))
                    changed = True
    return directed


def enforce_temporal_direction(directed, undirected):
    new_directed = set(directed)
    new_undirected = set(undirected)

    for x, y in list(undirected):
        if is_lag(x) and not is_lag(y):
            new_directed.add((x, y))
            new_undirected.discard((x, y))
            new_undirected.discard((y, x))
        elif is_lag(y) and not is_lag(x):
            new_directed.add((y, x))
            new_undirected.discard((x, y))
            new_undirected.discard((y, x))

    # sécurité
    new_directed = {
        (u, v) for (u, v) in new_directed
        if not (not is_lag(u) and is_lag(v))
    }

    return new_directed, new_undirected

def filter_invalid_edges(directed):
    clean = set()
    for u, v in directed:
        if not edge_allowed(u, v):
            continue
        clean.add((u, v))
    return clean


In [6]:
adj, sep_set = pc_skeleton(data, alpha=0.05, max_cond_set=2)
directed, undirected = orient_colliders(adj, sep_set)
directed = apply_meek_rules(directed, undirected)
directed, undirected = enforce_temporal_direction(directed, undirected)
directed = filter_invalid_edges(directed)

print("\nSTRUCTURE PC :\n")
for u, v in sorted(directed):
    print(f"{u} -> {v}")



STRUCTURE PC :

A021_lag -> A021
SEC01 -> A001
SEC01 -> A002
SEC01 -> A003
SEC01 -> A004
SEC01 -> A005
SEC01 -> A011
SEC01 -> A012
SEC01 -> A013
SEC01 -> A021
SEC01_lag -> A001
SEC01_lag -> A002
SEC01_lag -> A003
SEC01_lag -> A004
SEC01_lag -> A005
SEC01_lag -> A011
SEC01_lag -> A012
SEC01_lag -> A013
SEC01_lag -> A021
SEC02 -> A001
SEC02 -> A002
SEC02 -> A003
SEC02 -> A004
SEC02 -> A005
SEC02 -> A011
SEC02 -> A012
SEC02 -> A013
SEC02 -> A021
SEC02_lag -> A001
SEC02_lag -> A002
SEC02_lag -> A003
SEC02_lag -> A004
SEC02_lag -> A005
SEC02_lag -> A011
SEC02_lag -> A012
SEC02_lag -> A013
SEC02_lag -> A021
SEC03 -> A001
SEC03 -> A002
SEC03 -> A003
SEC03 -> A004
SEC03 -> A005
SEC03 -> A011
SEC03 -> A012
SEC03 -> A013
SEC03 -> A021
SEC03_lag -> A001
SEC03_lag -> A002
SEC03_lag -> A003
SEC03_lag -> A004
SEC03_lag -> A005
SEC03_lag -> A011
SEC03_lag -> A012
SEC03_lag -> A013
SEC03_lag -> A021


## Inference on unseen data

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


train_frac = 0.7
n_train = int(len(data) * train_frac)

train_data = data.iloc[:n_train]
test_data = data.iloc[n_train:]

parents_dict = {col: [] for col in data.columns}
for parent, child in directed:
    parents_dict[child].append(parent)

# Linear Gaussian CPDs
cpds = {}
for node in data.columns:
    parents = parents_dict[node]
    if len(parents) == 0:
        mu = train_data[node].mean()
        sigma = train_data[node].std()
        cpds[node] = {"type": "gaussian", "mu": mu, "sigma": sigma, "parents": []}
    else:
        X = train_data[parents].values
        y = train_data[node].values
        model = LinearRegression().fit(X, y)
        cpds[node] = {"type": "linear_gaussian", 
                      "model": model, 
                      "parents": parents}

predictions = pd.DataFrame(index=test_data.index, columns=assets)

for asset in assets:
    parents = cpds[asset]["parents"]
    if len(parents) == 0:
        predictions[asset] = cpds[asset]["mu"]
    else:
        X_test = test_data[parents].values
        model = cpds[asset]["model"]
        predictions[asset] = model.predict(X_test)

for asset in assets:
    mse = mean_squared_error(test_data[asset], predictions[asset])
    print(f"{asset} - Test MSE: {mse:.6f}")

print(predictions.head())

A001 - Test MSE: 0.000049
A002 - Test MSE: 0.000197
A003 - Test MSE: 0.000148
A004 - Test MSE: 0.000160
A005 - Test MSE: 0.000115
A011 - Test MSE: 0.000070
A012 - Test MSE: 0.000269
A013 - Test MSE: 0.000093
A021 - Test MSE: 0.000141
         A001      A002      A003      A004      A005      A011      A012  \
699  0.016734  0.011187  0.020699  0.015470  0.014423  0.000329  0.004186   
700  0.002732  0.001018  0.004012  0.002254  0.002787  0.006264  0.007032   
701  0.001216  0.000242  0.001950  0.000266  0.000831  0.000057 -0.000845   
702 -0.015151 -0.009355 -0.019171 -0.012960 -0.014103 -0.005586 -0.010154   
703 -0.010778 -0.005618 -0.014394 -0.009103 -0.009678 -0.012936 -0.018945   

         A013      A021  
699  0.003078  0.005416  
700  0.005776  0.001924  
701 -0.000667 -0.000477  
702 -0.008196 -0.010842  
703 -0.014945 -0.004218  


### Conditional Parameters

In [8]:
param_rows = []

for node, cpd in cpds.items():
    if cpd["type"] == "gaussian":
        param_rows.append({
            "node": node,
            "parent": None,
            "intercept": cpd["mu"],
            "coefficient": None,
            "sigma": cpd["sigma"]
        })

    elif cpd["type"] == "linear_gaussian":
        model = cpd["model"]
        parents = cpd["parents"]

        X = train_data[parents].values
        y = train_data[node].values
        y_hat = model.predict(X)
        sigma = np.sqrt(np.mean((y - y_hat) ** 2))

        param_rows.append({
            "node": node,
            "parent": "Intercept",
            "intercept": model.intercept_,
            "coefficient": model.intercept_,
            "sigma": sigma
        })

        for p, coef in zip(parents, model.coef_):
            param_rows.append({
                "node": node,
                "parent": p,
                "intercept": model.intercept_,
                "coefficient": coef,
                "sigma": sigma
            })

params_df = pd.DataFrame(param_rows)
params_df.head()


,node,parent,intercept,coefficient,sigma
0,MKT,None,-0.000221,NaN,0.009633
1,A001,Intercept,0.000045,0.000045,0.006833
2,A001,SEC01,0.000045,0.807720,0.006833
3,A001,SEC02_lag,0.000045,0.027902,0.006833
4,A001,SEC01_lag,0.000045,0.017751,0.006833
